# Testing the model we trained earlier with the wikipedia Arabic data dumps. 

# Importing Necessary Modules

In [1]:
import pandas as pd
import gensim as gs

import pyarabic.araby as araby
import pyarabic.number as number

import re

C:\Users\dell\Anaconda3\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# Loading up the data.

In [2]:
cleaned_articles_df_path = "F:\\Hackathon\\easyLearn\\cleaner 1 gb\\cleaner_one_gb_articles.csv"

In [3]:
articles = pd.read_csv(cleaned_articles_df_path,encoding='utf-16')

In [4]:
columns = ['id','title','text']
articles = articles[columns]
articles.head()

,id,title,text
0,7,ماء,الاكثر شيوعا في الارض غاز بخار ما سحاب الما ه...
1,8,ويكيبيديا:How to edit Arabic pages,تصنيف مساعده ويكيبيديا اسم الصفحه
2,30,ويكيبيديا:الميدان,محميه فهرسمجتمع ميدان رجا لا تعدل هذه الصفحه ...
3,38,رياضيات,في القرن الثالث كما تخيله رفائيل في لوحته الم...
4,46,الصفحة الرئيسية,لافهرس لاتحريرقسم مقدمه الصفحه الرئيسيه عنوان...


## Dropping Nan values.

In [5]:
articles.dropna()

,id,title,text
0,7,ماء,الاكثر شيوعا في الارض غاز بخار ما سحاب الما ه...
1,8,ويكيبيديا:How to edit Arabic pages,تصنيف مساعده ويكيبيديا اسم الصفحه
2,30,ويكيبيديا:الميدان,محميه فهرسمجتمع ميدان رجا لا تعدل هذه الصفحه ...
3,38,رياضيات,في القرن الثالث كما تخيله رفائيل في لوحته الم...
4,46,الصفحة الرئيسية,لافهرس لاتحريرقسم مقدمه الصفحه الرئيسيه عنوان...
5,62,إستونيا,ترجمه اليه تاريخ يوليو صم بلد اسم الدوله جمهو...
6,71,سنجاب,مصدر تاريخ يناير صندوق معلومات كائن ماهيه الت...
7,76,تلفاز,التلفاز وجمعه تلافز وتلفازات او التلفزه او ال...
8,77,A,ويكاموس حرف اسم همزهالف مفخمه رسم كتابه لاتين...
9,78,ويكيبيديا:بلاغ أخطاء وطلب ميزات,اختصار وب وب وب بلاغ الاخطا وطلب الميزات الجد...


# Loading up the trained model.

In [6]:
model_path = "F:\\Hackathon\\easyLearn\\Wikipedia Model - one_hour_and_33_minutes_trained_model\\one_hour_and_33_minutes_trained_model"

In [7]:
model = gs.models.doc2vec.Doc2Vec.load(model_path)

## Let's define some functions that will help us in viewing the recommender results in a better fashion.

In [8]:
def get_titles(sim_list, art_dataframe):
    titles = []
    for (tag,_) in sim_list:
        tag_id = int(tag.split('_')[1])
        titles.append(art_dataframe[art_dataframe['id'] == tag_id]['title'].values[0])
    return titles

In [9]:
def concatenate_list_data(list):
    result= ''
    for element in list:
        result += ' ' + str(element)
    return result

In [10]:
def get_texts(sim_list, art_dataframe):
    texts = []
    for (tag,_) in sim_list:
        tag_id = int(tag.split('_')[1])
        texts.append(art_dataframe[art_dataframe['id'] == tag_id]['text'].values[0])
    return texts

# Cleaning Methods

In [11]:
def remove_files(text):
    result = []
    regex = re.compile("\[.*?\]")
    temp_result = re.findall(regex, text)
    en_letters = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z'
             ,'A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Z','Y','Z']
    for string in temp_result:
        contains = any((c in string) for c in en_letters)
        if contains:
            result.append(string)
    for el in result:
        text = text.replace(el,' ')
    return text

In [12]:
def clean_some_chars(text):
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','&quot;','?','؟','!'
              ,"[","]","{","}","*",":","#","$","€","£","~","<",">","/","|","'",",",'=','(',')','+','•',';','&','–','♦','%'
             ,'»','»','·']
    
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ',' ',' '
               ," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," ",' ',' ',' ',' ',' ',' ',' ',' ',' ',' '
              ,' ',' ',' ']
    
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    text = text.replace('\'','')
    # removing numbers
    text = ''.join([i for i in text if not i.isdigit()])
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    
    #trim    
    text = text.strip()

    return text

In [13]:
def clean_english_chars(text):
    search = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z'
             ,'A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Z','Y','Z']
    for i in range(0, len(search)):
        text = text.replace(search[i],' ')
    return text

In [14]:
def remove_unnecessary_spaces(text):
    return re.sub(' +',' ',text)

In [15]:
def remove_non_arabic_letters(text):
    
    '''
    ALEF_MADDA       = u'\u0622' 
    ALEF_HAMZA_ABOVE = u'\u0623' 
    WAW_HAMZA        = u'\u0624' 
    ALEF_HAMZA_BELOW = u'\u0625' 
    YEH_HAMZA        = u'\u0626' 
    ALEF             = u'\u0627' 
    BEH              = u'\u0628' 
    TEH_MARBUTA      = u'\u0629' 
    TEH              = u'\u062a' 
    THEH             = u'\u062b' 
    JEEM             = u'\u062c' 
    HAH              = u'\u062d' 
    KHAH             = u'\u062e' 
    DAL              = u'\u062f' 
    THAL             = u'\u0630' 
    REH              = u'\u0631' 
    ZAIN             = u'\u0632' 
    SEEN             = u'\u0633' 
    SHEEN            = u'\u0634' 
    SAD              = u'\u0635' 
    DAD              = u'\u0636' 
    TAH              = u'\u0637' 
    ZAH              = u'\u0638' 
    AIN              = u'\u0639' 
    GHAIN            = u'\u063a' 
    TATWEEL          = u'\u0640' 
    FEH              = u'\u0641' 
    QAF              = u'\u0642' 
    KAF              = u'\u0643' 
    LAM              = u'\u0644' 
    MEEM             = u'\u0645' 
    NOON             = u'\u0646' 
    HEH              = u'\u0647' 
    WAW              = u'\u0648' 
    ALEF_MAKSURA     = u'\u0649' 
    YEH              = u'\u064a' 
    MADDA_ABOVE      = u'\u0653' 
    HAMZA_ABOVE      = u'\u0654' 
    HAMZA_BELOW      = u'\u0655' 
    LAM_ALEF                     = u'\ufefb' 
    LAM_ALEF_HAMZA_ABOVE         = u'\ufef7' 
    LAM_ALEF_HAMZA_BELOW         = u'\ufef9' 
    LAM_ALEF_MADDA_ABOVE         = u'\ufef5' 
    '''
    
    regex = re.compile(r'[\u0622\u0623\u0624\u0625\u0626\u0627\u0628\u0629\u062a\u062b\u062c\u062d\u062e\u062f\u0630\u0631\u0632\u0633\u0634\u0635\u0636\u0637\u0638\u0639\u063a\u0640\u0641\u0642\u0643\u0644\u0645\u0646\u0647\u0648\u0649\u064a\u0653\u0654\u0655\ufefb\ufef7\ufef9\ufef5]')
    # removing Arabic letters from the text and storing the result in the varialbe: unwanted_str .
    unwanted_str = regex.sub(' ',text)
    # Creating a list containing all of the unwanted characters, letters and symbols.
    unwanted_list_of_strs = list(unwanted_str.replace(" ", ""))
    # Cleaning the unwanted list of characters out of the text
    for i in range(0, len(unwanted_list_of_strs)):
        text = text.replace(unwanted_list_of_strs[i], " ")
    
    text = remove_unnecessary_spaces(text)
    
    return text

In [16]:
def concatenate_list_into_string(lis_strs):
    result = ""
    for el in lis_strs:
        result += " " + el
    return result

In [17]:
def remove_single_letters(text):
    words = text.split(' ')
    waw = 'و'
    for word in words:
        if len(word.strip()) == 1:
            if word != waw:
                words.remove(word)
    text = concatenate_list_into_string(words)
    return text

In [18]:
def clean_text(text):
    # removing files
    text = remove_files(text)
    # removing some unuseful chars
    text = clean_some_chars(text)
    # removing english chars
    text = clean_english_chars(text)
    # removing tashkeel
    text = araby.strip_tashkeel(text)
    # removing longation
    text = araby.strip_tatweel(text)
    # removing unwanted spaces
    text = remove_unnecessary_spaces(text)
    # removing non-arabic characters
    text = remove_non_arabic_letters(text)
    # removing single unwanted letters
    text = remove_single_letters(text)
    # returning result
    return text    

# Now, we can start testing the model.

## 1st example
### This is an Arabic article that gives an introduction about Artificial Integlligence.
You can find this article here (source): https://www.syr-res.com/article/12382.html

In [19]:
artificial_intelligence_intro = """<span property="articleBody">
			
			يختلف العلماء والفلاسفة في تعريف الذكاء البشري، لكنَّ أهم التعريفات الموجودة إلى الآن تُعرّف الذكاء على أنه القدرة على التعلم، والتكيف مع البيئة، والتفكير المجرد.
<br />
يُمكِّن الذكاءُ البشرَ من تحقيق أهدافهم وممارسة حياتهم بشكلٍ سلس؛ فمثلاً بما أن البشر يملكون القدرة على التفكير المجرد فإنهم أيضاً قادرون على تطوير اللغات للتواصل بين بعضهم، وبما أنّ البشر يمتلكون القدرة على التعلم فإنهم يستفيدون من خبرات الماضي في التعامل مع الحاضر كالطفل الذي يتجنب الأشياء الساخنة لأنها أحرقته مرةً.
<br />
يُمكّنهم الذكاء أيضاً من التأقلم مع البيئة المحيطة بهم، فالذّكاء البشري هو الذي يدفع البشر لارتداء معاطفَ في الشتاء، ويدفعهم لتجنّب المخاطر، ويدفع الطفل لتعلّم أساليبَ جديدةٍ للتعلم في المدرسة كي يتأقلم مع بيئته. 
<br />

<br />
هذا هو الذكاء البشري -وبشكلِ مبسَّط جداً- فماذا عن الذكاء الصنعي؟
<br />
يُقسَم الذكاءُ الصنعي من حيثِ التعريف وطريقةِ العلم للوصول إليه إلى نوعين: محاولةُ جعلِ الآلةِ ذكيةً كالبشر، ومحاولةِ جعل الآلة تتصرف تصرفاتٍ ذكية، وهنا يكمن فرقٌ ضخمٌ بين الاثنين؛ إذ أنه من الممكن للآلة أن تتصرف بشكل ذكيٍّ دون أن تكون ذكيةً بنفسها، وأن تكون قادرةً على تعلم مهامٍ جديدةٍ تماماً. فمثلاً نظام الطيار الآلي هو نظامٌ يَستخدم الذكاء الصنعي ويَتصرّف بذكاء، إذ أنّه يقرأ معطياتِ البيئة المحيطة ويقرر مسارَ الطائرة وارتفاعَها وأموراً أُخرى، لكنّه ليس ذكيّاً بالمعنى الحرفي؛ فلا يُمكن مثلاً أن نتوقع من هذا النظام أن يبدأ فجأةً بقراءة الشعر أو تعلم فيزياء الكم لأنه صُمِّمَ ليتصرَّف بذكاءٍ لا ليكون ذكياً.
<br />

<br />
في سلسلة المقالات هذه سوف نخوض في الكثير من المسائل، منها ما يتعلق بكيفية جعل الآلة ذكيةً، ومنها بكيفية جعل الآلة تتصرف بذكاء. وسنتطرّق أيضاً خلالها إلى أقسامه المختلفة لنناقش ما هو تعلم الآلة، وما هي الشبكات العصبونية في الحواسيب، وكيف يُمثَّل المنطق، وكيف تُمثَّل المسائل التي تحتاج لتفكيرٍ مجرّدٍ كاللغات.
<br />
 وسنبدأ بالمهمة الأسهل نسبياً: جعل الآلة تتصرف بشكلٍ ذكي.
<br />

<br />
قبل أن نقوم بذلك، نحن بحاجةٍ لإيجاد طريقةٍ لإفهام الآلة ما هي المشكلة التي نحاول حلها، أو بعبارةٍ أُخرى؛ علينا إيجاد طريقةٍ لنعبّر عن المسائل بشكلٍ برمجيٍّ ما.
<br />

<br />
أولى هذه الطرق هي بالتعبير عن المسألة عن طريق بناء شجرةٍ تُمثّل المسألة تتكوّن هذه الشجرة من عُقَدٍ تتصل مع بعضها بأغصانٍ أو أضلاعٍ تُمثّل علاقةً ما بين العقدة والأُخرى التي تليها. لنأخذْ مثلاً أُحجيةَ ٨، هذه الأحجيةُ عبارةٌ عن ٨ مربعاتٍ صغيرةٍ من الممكن تحريكُها، وهدفُ اللعبةِ أن يتمَّ ترتيب الأرقام بشكلٍ تصاعديٍّ (أو تنازلي).
<br />

<br />
<a target='_blank' href='http://www.syr-res.com/pic_ret.php?id=23738&b=1'><img title='' class='double-border' style='max-width: 677px; width: 100%;' src='http://www.syr-res.com/pic_ret.php?id=23738&b=1'></a><br/><small class='inline-copyright'>Image: http://www.aiai.ed.ac.uk/~gwickler/eightpuzzle-inf.html</small>
<br />

<br />
لنستطيعَ جعلَ الآلة تفهمُ كيف من الممكن أن تحُلُّ هذه اللعبة؛ سنعبِّر عنها بشجرةٍ ذروتُها هي الحالة الأولى التي نجد الأحجية عليها، والمستوى الأول من الفروع يُمثِّل جميع الحركات الممكنة، والمستوى التالي يمثّل جميع الحركات الممكنة من المستوى الذي يسبقه، وهكذا دواليك.
<br />

<br />
<a target='_blank' href='http://www.syr-res.com/pic_ret.php?id=23739&b=1'><img title='' class='double-border' style='max-width: 217px; width: 100%;' src='http://www.syr-res.com/pic_ret.php?id=23739&b=1'></a><br/><small class='inline-copyright'>Image: http://www.wou.edu/las/cs/csclasses/cs160/VTCS0/AI/Lessons/GamePlaying/gametree.gif</small>
<br />

<br />
كمثالٍ آخر عن تمثيل المُشكلات في الحاسوب، نجد برامج الخرائط التي تساعد المستخدِمين على إيجاد الطريق لمُرادهم بأسرعِ وقتٍ ممكن. تعبّر هذه البرامج عن المدن على شكلِ عُقدٍ تتصل مع بعضها بأغصانٍ أيضاً، تمثّل هذه العقد نقاطاً في المدينة، وتمثل الأغصان طرقاتٍ، وما على الحاسوب الآن إلا أن يسير من بداية الطريق مُتّبعاً خوارزميةً ما لإيجاد الحل الأمثل.
<br />
تمثّل خوازرميات البحثِ موضوعاً هاماً في الذكاء الصنعي، وذلك لأنه يُستَخدم لحلِّ الكثير من المسائل. من هذه المسائل مسألةُ وضعِ ثمانية قطعِ وزراءٍ في الشطرنج على الرقعة دون أن يأكل أحدهم الآخر، أو ببساطةٍ إعادة ترتيب قطعٍ فوق بعضها البعض وفق ترتيبٍ مُعيَّن.
<br />

<br />
ماذا تعني هذه المسائل؟
<br />
إن عبَّرنا عن مجموعةِ التنقُّلات المُمكنة بالشجرة سابقة الذكر، فإنَّ المسائل ستعني أننا نبدأُ من وضعٍ بدئيٍّ (عقدةٍ في شجرة)، ونريد الوصول إلى وضعٍ نهائي (عقدةٍ أخرى) عبر سلسلةٍ من التنقلات (الأضلاع)، وإنّ إيجاد السلسلة التي تسمح بالوصول إلى الوضع النهائي المرغوب يكون عبر البحث.
<br />
فإذا أردنا الانطلاق من A واكتشافَ العُقد الباقية. سيكون لدينا وسيلتان للتنقل:
<br />
- أن نخوضَ كلَّ فرعٍ إلى نهايته ثم ننتقل للفرعِ الآخرِ باحثينَ عن الهدف. أي أننا سوف ننطلق وفق التسلسل التالي:
<br />
A B D H I E J K C F L M G N O، وتُدعى هذه الطريقة بالبحث وفق العُمق أولاً (Depth First Search / DFS) وتَستخدم بنيةً في الحاسوب تدعى بالمكدّس Stack، إذ أنّ آخر عقدة تم إدخالها هي أول عقدة سوف يتم اكتشافهاLast In First Out (LIFO).
<br />

<br />
<a target='_blank' href='http://www.syr-res.com/pic_ret.php?id=23740&b=1'><img title='' class='double-border' style='max-width: 620px; width: 100%;' src='http://www.syr-res.com/pic_ret.php?id=23740&b=1'></a><br/><small class='inline-copyright'>Image: http://aima.cs.berkeley.edu/</small>
<br />

<br />
- أن نكتشف جميع العقد التي تقع في مستوىً واحد، ثم ننتقلُ إلى المستوى التالي، وهكذا...
<br />
أي أننا سوف ننطلق وفق التسلسل التالي:
<br />
A B C D E F G، وتُدعى هذه الطريقة بالبحث وفق العرض أولاً Breadth First Search BFS وتَستخدم في الحاسوب بنيةً تدعى بالصف Queue، إذ أنّ العقدة التالية التي سيتم اكتشافها هي أول عقدة تم إدخالها فيه First In First Out (FIFO).
<br />

<br />
<a target='_blank' href='http://www.syr-res.com/pic_ret.php?id=23741&b=1'><img title='' class='double-border' style='max-width: 622px; width: 100%;' src='http://www.syr-res.com/pic_ret.php?id=23741&b=1'></a><br/><small class='inline-copyright'>Image: http://aima.cs.berkeley.edu/</small>
<br />

<br />
تُشكّل هاتان الطريقتان ما يدعى بخوارزميات البحث العمياء Uninformed Search Algorithms لأنها لا تملِكُ أيَّ معلوماتٍ عن كيفيةِ عملِ المسألة المطروحة. كلُّ ما تقوم به هو توليدُ خطواتٍ متلاحقةٍ والتمييزُ بين نقطة الانطلاق والنقطة التي نرغبُ بالوصول إليها.
<br />

<br />
إلا أنَّ هناك خوارزمياتٍ يُمكِنها معرفة إن كانت إحدى العقدِ الواقعة بين البداية والهدف؛ واعدةً أكثرَ من غيرِها، أي احتمالُ الوصولِ عبرَها بشكلٍ جيّدٍ إلى الهدف، أفضلُ من غيره لأنّها تُدرك معلوماتٍ إضافيةً عن المسألة التي يتم حلها وكيفية عملها.
<br />
تُدعى هذه الخوارزميات بخوارزميات البحث المُوجَّهة باستخدام الاستدلال، فكيف تعمل هذه الخوارزميات؟
<br />
تقوم هذه الخوارزميات باختيار العقدة التالية الأقل تكلفة، ويتم حساب هذه التكلفة عبر تابعِ تقديرٍ يعتمدُ على الاستدلال ويُمثّلُ التكلفةَ التقديريةَ للوصول من هذه العقدة إلى العقدة الهدف، كأن يقوم شخصٌ ما موجودٌ في رومانيا، بتقديرِ أنَّ أقلَّ تكلفةٍ للوصول من مدينة آراد إلى مدينة بوخارست هي تكلفةُ عبور الطريق المستقيم بين المدينتين.
<br />
تُدعى إحدى أشهرُ هذهِ الخوارزميات بخوارزمية A* search، التي تقوم بتقييم العُقد عبرِ جمعِ تكلفةِ الوصول إلى هذه العقدة، مضافاً إليها التكلفةَ التقديريَّةَ للوصول منها إلى الهدف. ثُمَّ تختار العقدةَ ذاتَ التكلفةِ الأقلّ، ليتم اكتشافُها تالياً، بدلاً من أن يكون اختيارُ العقدةِ التالية معتمِداً تماماً على بنية الشجرة كما في خوارزميات البحث العمياء.
<br />
نلاحظ في المثال التالي أنَّ العقدة التي تَقرَّرَ اكتشافُها بعد العقدة الأولى Arad هي العقدة ذات التكلفة الأقل Sibiu، ثم تمَّ اختيارُ العقدة ذات التكلفة الأقل وهي Fagaras وهكذا.
<br />

<br />
<a target='_blank' href='http://www.syr-res.com/pic_ret.php?id=23742&b=1'><img title='' class='double-border' style='max-width: 470px; width: 100%;' src='http://www.syr-res.com/pic_ret.php?id=23742&b=1'></a><br/><small class='inline-copyright'>Image: http://aima.cs.berkeley.edu/</small>
<br />

<br />
لكي نأخذَ مثالاً آخرَ عن هذه الخوارزميات سنعود إلى اللوحة التي تحتوي على الأرقام من 1 إلى 8 وعلينا ترتيبها، فيمكننا اعتبار تابع الاستدلال:
<br />
1- يُمثّلُ في كلِّ حالةٍ، ما إذا كان الرقم يقع في مكانه الصحيح (وعندها القيمة 0)، أو في مكانٍ خاطئ (وعندها القيمة 1)، ثُمَّ نجمع هذه القيم، وهذا ما يدعى بتابع أولوية هامنغ.
<br />
2- يُمثل في كل حالةٍ عددَ الخطوات المطلوبة لكلِّ رقمٍ ليصبحَ في موقعه الصحيح ثمّ نجمع هذه القيم، وهذا ما يدعى بتابع أولوية مانهاتن.
<br />
ثمّ يتم تحديد الخطوة التالية تبعاً لهذا التابع بالإضافة لتكلفة الوصول لكل حالة.
<br />

<br />
<a target='_blank' href='http://www.syr-res.com/pic_ret.php?id=23743&b=1'><img title='' class='double-border' style='max-width: 660px; width: 100%;' src='http://www.syr-res.com/pic_ret.php?id=23743&b=1'></a><br/><small class='inline-copyright'>Image: https://www.cs.princeton.edu/courses/archive/spr10/cos226/assignments/8puzzle.html</small>
<br />

<br />
بعد هذه الجولة التعريفية في عالم الذكاء الصنعي، والتعرُّفِ على كيفية تمثيل المسائل وبعض خوارزميات الحل، سنتابع في مقالاتنا القادمة التعمّقَ في أقسامه ومجالاته وتطبيقاته في حياتنا اليومية.
<br /><br />المصادر
<br />"""

Here's the text of the article (with html markup and web links)

In [20]:
artificial_intelligence_intro

'<span property="articleBody">\n\t\t\t\n\t\t\tيختلف العلماء والفلاسفة في تعريف الذكاء البشري، لكنَّ أهم التعريفات الموجودة إلى الآن تُعرّف الذكاء على أنه القدرة على التعلم، والتكيف مع البيئة، والتفكير المجرد.\n<br />\nيُمكِّن الذكاءُ البشرَ من تحقيق أهدافهم وممارسة حياتهم بشكلٍ سلس؛ فمثلاً بما أن البشر يملكون القدرة على التفكير المجرد فإنهم أيضاً قادرون على تطوير اللغات للتواصل بين بعضهم، وبما أنّ البشر يمتلكون القدرة على التعلم فإنهم يستفيدون من خبرات الماضي في التعامل مع الحاضر كالطفل الذي يتجنب الأشياء الساخنة لأنها أحرقته مرةً.\n<br />\nيُمكّنهم الذكاء أيضاً من التأقلم مع البيئة المحيطة بهم، فالذّكاء البشري هو الذي يدفع البشر لارتداء معاطفَ في الشتاء، ويدفعهم لتجنّب المخاطر، ويدفع الطفل لتعلّم أساليبَ جديدةٍ للتعلم في المدرسة كي يتأقلم مع بيئته. \n<br />\n\n<br />\nهذا هو الذكاء البشري -وبشكلِ مبسَّط جداً- فماذا عن الذكاء الصنعي؟\n<br />\nيُقسَم الذكاءُ الصنعي من حيثِ التعريف وطريقةِ العلم للوصول إليه إلى نوعين: محاولةُ جعلِ الآلةِ ذكيةً كالبشر، ومحاولةِ جعل الآلة تتصرف تصرفاتٍ ذكي

Let's clean the text of this article and see what we get.

In [21]:
artificial_intelligence_intro = clean_text(artificial_intelligence_intro)

In [22]:
artificial_intelligence_intro

'  يختلف العلما والفلاسفه في تعريف الذكا البشري لكن اهم التعريفات الموجوده الي الان تعرف الذكا علي انه القدره علي التعلم والتكيف مع البيئه والتفكير المجرد يمكن الذكا البشر من تحقيق اهدافهم وممارسه حياتهم بشكل سلس فمثلا بما ان البشر يملكون القدره علي التفكير المجرد فانهم ايضا قادرون علي تطوير اللغات للتواصل بين بعضهم وبما ان البشر يمتلكون القدره علي التعلم فانهم يستفيدون من خبرات الماضي في التعامل مع الحاضر كالطفل الذي يتجنب الاشيا الساخنه لانها احرقته مره يمكنهم الذكا ايضا من التاقلم مع البيئه المحيطه بهم فالذكا البشري هو الذي يدفع البشر لارتدا معاطف في الشتا ويدفعهم لتجنب المخاطر ويدفع الطفل لتعلم اساليب جديده للتعلم في المدرسه كي يتاقلم مع بيئته هذا هو الذكا البشري وبشكل مبسط جدا فماذا عن الذكا الصنعي يقسم الذكا الصنعي من حيث التعريف وطريقه العلم للوصول اليه الي نوعين محاوله جعل الاله ذكيه كالبشر ومحاوله جعل الاله تتصرف تصرفات ذكيه وهنا يكمن فرق ضخم بين الاثنين اذ انه من الممكن للاله ان تتصرف بشكل ذكي دون ان تكون ذكيه بنفسها وان تكون قادره علي تعلم مهام جديده تماما فمثلا نظام الطيار 

In [23]:
tokens = artificial_intelligence_intro.split()

new_vector = model.infer_vector(tokens)
sims = model.docvecs.most_similar([new_vector]) #gives you top 10 document tags and their cosine similarity

sims

[('SENT_300222', 0.7973574995994568),
 ('SENT_7', 0.7874107360839844),
 ('SENT_1250', 0.7810664176940918),
 ('SENT_301167', 0.7808771729469299),
 ('SENT_209125', 0.7800158262252808),
 ('SENT_585438', 0.7793781757354736),
 ('SENT_1343', 0.779096245765686),
 ('SENT_436578', 0.7784536480903625),
 ('SENT_1359', 0.7773736119270325),
 ('SENT_143123', 0.7760809659957886)]

### Getting titles of the resulted articles.

In [24]:
get_titles(sims,articles)

['شبكات استشعار لاسلكية',
 'ماء',
 'ابن خلدون',
 'اضطراب نقص الانتباه مع فرط النشاط',
 'شلل دماغي',
 'ويكيبيديا:الميدان/أرشيف/سياسات/06/2009',
 'برلين',
 'ويكيبيديا:الميدان/أرشيف/سياسات/11/2008',
 'ويكيبيديا:حقوق التأليف والنشر',
 'سرطان المبيض']

### Getting texts of the resulted articles.

In [25]:
get_texts(sims,articles)

[' شبكات الاستشعار اللاسلكيه هي عباره عن مجموعه من اجهزه استشعار الاستشعار التي تستخدم في نقل او متابعه ظاهره فيزيائه او كميائيه محدده كالحراره الرطوبه الاهتزاز الضو الخ ومن ثم نقل المعلومات عن الظاهره لاسلكيا الي مركز معالجه البيانات للاستفاده منها دون الحاجه لتواجد الانسان في مكان الظاهره الفيزيائيه نظره عامه وقد بدات شبكه الاستشعار اللاسلكيه باعتبارها واحده من افضل التكنولوجيات الناشئه في القرن الحادي والعشرين تتطور بوتيره متسارعه في السنوات العشر الماضيه وقد تم القيام بالكثير من الابحاث لتحسينها في مختلف الجوانب بما في ذلك بنائها وانظمه تشغيل العقد وبروتوكولات التوجيه وجمع البيانات ودمجها واليه تحديد الموقع ومزامنه الوقت وما الي ذلك وعلاوه علي ذلك ظهرت اعداد كبيره من التطبيقات الواعده وتم نشرها في مناطق جغرافيه مختلفه مثل حمايه الهياكل الاساسيه والاستكشاف العلمي والمراقبه العسكريه ومراقبه حركه المرور والسيطره وامن التعدين والنقل البحري وحمايه البيئه وتتبع الاجسام والشؤون العسكريه وما الي ذلك مع وسائل الراحه التي قدمتها شبكه الاستشعار اللاسلكيه وقد اثرت في حياتنا وتغيرت الي حد كبير 

## 2nd example
### This is an Arabic article that introduces computer science and software engineering in a comparing manner.
You can find this article here (source): https://www.syr-res.com/article/11036.html

In [26]:
cs_se_article = """"ما المقصود بهندسة البرمجيات؟ يعجز الكثيرُ من خريجي الجامعات ممن درسوا علوم الحاسب، عن التفريق بين علوم الحاسب وهندسة البرمجيات. علوم الحاسب بالإنجليزية هي: (Computer Science) أما هندسة البرمجيات فهي: (Software Engineering).
<br />
<br />يشترك علم الحاسب مع هندسة البرمجيات في المهارات البرمجية، لهذا قبل الخوض في هندسة البرمجيات دعونا نتعرف أولاً على مصطلح البرمجة أو ما يعرف بتطوير البرمجيات (Software Development)، يصف أحد الطلاب مراحلَ تطوير البرمجيات كالآتي:
<br />"أثناء تصميم البرنامج، أكون مصمماً. أثناء تصميم الواجهة، أكون فناناً. أثناء مراحل كتابة كود البرنامج، أكون حِرَفِيّاً. وأثناء مرحلة التصحيح، أكون فظاً عديمَ المشاعر!"
<br />لا يزالُ الجدلُ قائماً حولَ ما إذا كان تطوير البرمجيات علماً أم فنّاً، قبل ثلاثين سنة كَتَب العالم دونالد نوث (Donald Knuth) سلسلةً من سبعةِ مجلدات حملت عُنوان: "فنُّ البرمجة بالحاسوب" (The Art of Computer Programming)، حيث تراوح عددُ صفحات أول ثلاثة مجلداتٍ حَوالي 2،200 صفحة، ما يجعلُ العدد الإجمالي لكامل صفحات السلسلة ما يقارب الـ 5،000.
<br />إذا كان هذا العدد موجّهاً للبرمجة فقط، فكم تتوقع سيلزمُ من الصفحات للحديث عن علوم الحاسب؟
<br />
<br />•هل البرمجة علمٌ أم فن؟
<br />يُشير المؤيدون لمنظور البرمجة كـ "فن" إلى الجانب الجمالي لتطويرالبرمجيات، ويَرون أن العلوم لا تمتلك السلاسة والحرية الإبداعية التي يتمتع بها الفن.
<br />في المقابل، يحتج الداعمون لمنظورالبرمجة كـ "علم" بأنّ عدمَ اعتماد المنهج العلمي أثناء البرمجة يزيدُ من احتمالية وقوع الأخطاء، ما يقلّلُ الثقة في البرنامج.
<br />في الحقيقة، كلا النظريتين ناقصتان، وتطرحان الأسئلة الخاطئة؛ البرمجة (أو تطوير البرمجيات) هي علمٌ وفنٌ في آنٍ واحد. البرمجةُ بحدِّ ذاتها تُمارس من قِبَل الهُواة والمحترفين على حدٍّ سواء لذلك فالسؤال الذي يجب أن يُطرح هو: "كيف تكون البرمجة المحترفة؟"
<br />يكمن الجواب في "هندسة البرمجيات" (Software Engineering)؛ هندسة البرمجيات هي طريقة المحترفين في عملية تطوير البرامج.
<br />يستعمِلُ البعض وصفَ "البرمجة" بدلَ "هندسة البرمجيات" كنوعٍ من التبسيط، في اعتقادٍ منهم أنّ هندسة البرمجيات ما هي إلا أسلوبٌ مرتبٌ ومنظّمٌ لكتابة الأسطر البرمجية، ولكنّ هذا خطأ؛ فتطوير التطبيقات البرمجية لم يعد مقتصراً على معرفة علوم الحاسب فقط، ولهذا وُجِدت هندسة البرمجيات، لإضافة معلوماتٍ أُخرى خارج نطاق علوم الحاسب.
<br />ولكن ما هي الهندسة تحديداً؟
<br />بحسب تعريف القاموس: الهندسةُ هي تطبيقُ قواعدَ رياضيةٍ وعلميةٍ للوصول إلى نتائجَ عملية، وفي مجال تطوير البرمجيات، يعني هذا تطبيقَ خوارزمياتٍ مبنيةٍ علمياً وعلى أساسٍ رياضيٍّ لتصميمِ برامجَ وخدماتِ ويب وغيرها..
<br />
<br />هندسةٌ أم علم؟
<br />نأتي أخيراً إلى سؤالنا المهم وهو، ما علاقة علوم الحاسب بهندسة البرمجيات؟
<br />التباين بين العلوم والهندسة هو نفسُه مهما اختلفت المجالات، دعونا نتعرف أولاً على الفرق بين الهندسة والعلوم بشكلٍ عام؛ في ميدان العلوم، يبحثُ العالِمُ عن الحقيقة عن طريق تجربة الفرضيات والخروج باستنتاجاتٍ تُساهم في توسيع المعرفة العلمية. أما في ميدان الهندسة، فيكون هدفُ المهندس البحثَ عمّا هو مفيد، وعن كيفية تطبيق المعارف المكتسبة لحل مشكلاتٍ من الواقع. على العلماء أن يكونوا على اطّلاعٍ بآخر الأبحاث في مجالهم العلمي، في حين أنه على المهندسين أن يكونوا على درايةٍ بالمعارف الموثوقة التي أثبتت فاعليتها.
<br />باستطاعة الأشخاص الذين اختاروا الاختصاصات العلمية أن يتخصصوا أكثرَ في مجالهم، بينما يُطلَب من المهندسين معرفةً موسعةً بكلِّ ما يتعلّقُ بالعوامل التي ستؤثر على المنتج النهائي.
<br />لا يحتاج العلماء لقوانينَ تُنظِّم عملهم لأنهم يتعاملون مع بعضهم فقط، في حين يحتاج المهندسون لمثل هذه القوانين كونَهم يتعاملون مع أشخاصٍ عاديين.
<br />يُعِدُّ اختصاصُ الهندسةِ الطالبَ للحياة العمليّة، بينما يفتحُ الاختصاصُ العلمي الطريقَ أمام الطالب للاستمرار في المجال البحثي الأكاديمي ولا يعطي الشيء الكثير لبدء حياة عمليّة.
<br />
<br />العواقب التي يدفع الطالب ثمنها نتيجة الخلط بين المجالين:
<br />تمنحُ معظم الجامعات طلابها درجاتٍ في علوم الحاسب وتتوقع منهم الحصول على وظائف في تطوير البرمجيات، يقومون فيها بحلّ معضلات من الواقع؛ في حين أن نسبةً قليلةً من الطلاب يتوجهون نحو الدراسات العليا والمجال البحثي. مما يضع الطلابَ أمامَ واقعٍ تكنولوجي متضارب؛ فبينما هم يمتلكون درجاتٍ علمية فهم يقومون بوظائفَ تكون مقتصرةً على المهندسين، وهذا دون الحصول على تدريب مسبق. الأمر كما لو أنّك تطلب من طالب دكتوراة في الفيزياء بناءَ محرّكٍ كهربائي للاستهلاك العام؛ قد يفهمُ عالم الفيزياء قوانينَ الكهرباء أكثرَ من المهندس، ولكنّه ليس مدرباً على بناء منتجاتٍ لحلّ مشكلاتٍ من الواقع، حيث تقتصر خبرته على بناء نماذجَ مخبريّةٍ لا أكثر (prototypes)، ربما سيعملُ المُنتج، ولكنّه لن يكون متيناً بما فيه الكفاية للعمل خارج المختبر.
<br />تحدثُ حالاتٌ مُشابهة في مجال تطوير البرمجيات، حيث يطور خريجو علوم الحاسب برامجَ ذات بنيةٍ ضعيفةٍ وغيرِ آمنة، فيركّزون على أمورٍ ثانوية مُهملينَ أموراً أكثرَ أهمية في بناء المنتج، فقد يقضي خريجُ علوم الحاسب مدة أسبوعين في صقل وتعديل خوارزمية، بدلاً من قضاء نفس المدة في تعلّم استخدامِ مكتبةٍ برمجيةٍ جديدة. باختصار، يحتاج طالب علوم الحاسب لسنواتٍ إضافية للتدريب على المجال العمليّ لاكتساب الخبرة الضرورية. وليس هذا هو السبب الوحيد، فتطور سوق العمل أصبح يتسارع بوتيرةٍ أكبر من المنظومة الأكاديمية.
<br />
<br />خلاصة: هندسة البرمجيات ليست علوم حاسب:
<br />في خضمّ واقعنا المتسارع تكنولوجياً، تتزايد حاجتنا لبرامج عالية الكفاءة، ومع أهمية هذه البرامج في حياتنا، وآخر المُستجدات العلمية المتعلقة بطرق تطويرها، تظهر الحاجة لمختصين ليس في تطوير وإنتاج هذه البرامج فحسب، بل في تصميمها وبناءها واختبارها وصيانتها أيضاً. لهذا يمكننا القول أن هندسة البرمجيات لم تعد جزءاً من علوم الحاسب.
<br />
<br />---------------------------------------------------------------
<br />
<br />المصادر:
<br />
<br />[1] Professional Software Development: Shorter Schedules، Higher Quality Products، More Successful Projects، Enhanced Careers. - Steve McConnell.
<br /><a style='font-weight: bold; color: #23B0FC' target=_blank href='https://drive.google.com/open?id=0B2qXg8qQzBCQWENleDRFaGdrYVE'>هنا</a>
<br />
<br />[2] Software Engineering Programmes are Not Computer Science Programmes - David Lorge Parnas.
<br /><a style='font-weight: bold; color: #23B0FC' target=_blank href='https://drive.google.com/open?id=0B2qXg8qQzBCQVGZJby1vZDJhZFE'>هنا</a>			</main>
			</span>"""

Here's the text of the article (with html markup and web links)

In [27]:
clean_text(cs_se_article)

' ما المقصود بهندسه البرمجيات يعجز الكثير من خريجي الجامعات ممن درسوا علوم الحاسب عن التفريق بين علوم الحاسب وهندسه البرمجيات علوم الحاسب بالانجليزيه هي اما هندسه البرمجيات فهي يشترك علم الحاسب مع هندسه البرمجيات في المهارات البرمجيه لهذا قبل الخوض في هندسه البرمجيات دعونا نتعرف اولا علي مصطلح البرمجه او ما يعرف بتطوير البرمجيات يصف احد الطلاب مراحل تطوير البرمجيات كالاتي اثنا تصميم البرنامج اكون مصمما اثنا تصميم الواجهه اكون فنانا اثنا مراحل كتابه كود البرنامج اكون حرفيا واثنا مرحله التصحيح اكون فظا عديم المشاعر لا يزال الجدل قائما حول ما اذا كان تطوير البرمجيات علما ام فنا قبل ثلاثين سنه كتب العالم دونالد نوث سلسله من سبعه مجلدات حملت عنوان فن البرمجه بالحاسوب حيث تراوح عدد صفحات اول ثلاثه مجلدات حوالي صفحه ما يجعل العدد الاجمالي لكامل صفحات السلسله ما يقارب ال اذا كان هذا العدد موجها للبرمجه فقط فكم تتوقع سيلزم من الصفحات للحديث عن علوم الحاسب هل البرمجه علم ام فن يشير المؤيدون لمنظور البرمجه فن الي الجانب الجمالي لتطويرالبرمجيات ويرون ان العلوم لا تمتلك السلاسه والحريه الابداعيه ال

Let's clean the text of this article and see what we get.

In [28]:
cs_se_article = clean_text(cs_se_article)

In [29]:
tokens2 = cs_se_article.split()

new_vector2 = model.infer_vector(tokens2)
sims2 = model.docvecs.most_similar([new_vector2]) #gives you top 10 document tags and their cosine similarity

sims2

[('SENT_1250', 0.7589057683944702),
 ('SENT_1359', 0.7514535784721375),
 ('SENT_1249', 0.7481476068496704),
 ('SENT_1298', 0.7426693439483643),
 ('SENT_1299', 0.7415467500686646),
 ('SENT_1318', 0.7389407157897949),
 ('SENT_1343', 0.7363722920417786),
 ('SENT_1201', 0.7354879379272461),
 ('SENT_7', 0.7350192666053772),
 ('SENT_1363', 0.7349768877029419)]

### Getting titles of the resulted articles.

In [30]:
get_titles(sims2,articles)

['ابن خلدون',
 'ويكيبيديا:حقوق التأليف والنشر',
 'عمان (مدينة)',
 'هلسنكي',
 'برمجيات حرة',
 'جبر',
 'برلين',
 'شعر (أدب)',
 'ماء',
 'الصلاة في الإسلام']

### Getting texts of the resulted articles.

In [31]:
get_texts(sims2,articles)

[' تعظيم تاريخ نوفمبر صندوق معلومات شخص اسم ابن خلدون الصوره حجم الصوره تعليق الصوره تمثال لابن خلدون في قلب العاصمه ساحه الاستقلال في تونس الجنسيه العقيده مسلم المذهب المذهب المالكي تاريخ الولاده يوم الاربعا رمضان الموافق مايو تاريخ الوفاه الجمعه رمضان الموافق مارس ابن خلدون هو عبد الرحمن بن محمد ابن خلدون ابو زيد ولي الدين الحضرمي الاشبيلي ولد في تونس وشب فيها وتخرج من جامعه الزيتونه ولي الكتابه والوساطه بين الملوك في المغرب العربي بلاد المغرب و الاندلس ثم انتقل الي مصر حيث قلده السلطان برقوق قضا المالكيه ثم استقال من منصبه وانقطع الي التدريس والتصنيف فكانت مصنفاته من اهم المصادر للفكر العالمي من اشهرها كتاب العبر وديوان المبتدا والخبر في معرفه ايام العرب والعجم والبربر ومن عاصرهم من ذوي السلطان الاكبر تاريخ ابن خلدون ابن خلدون مؤرخ من شمال افريقيا تونس مدينه تونسي المولد الاندلس اندلسي الحضارمه حضرمي الاصل عاش بعد تخرجه من جامعه الزيتونه في مختلف مدن شمال افريقيا حيث رحل الي بسكره و غرناطه و بجايه و تلمسان كما توجه الي مصر حيث اكرمه سلطانها الظاهر سيف الدين برقوق الظاهر برقوق ولي في

## The model is doing his job properly :D

## And, it's ready for deployment in any kind of app (web, mobile and desktop)

### Now, that we have completed training and testing the model we can use the gensim function Doc2Vec.delete_temporary_training_data() function.

Description of the what does this function do:
Discard parameters that are used in training and score. Use if you’re sure you’re done training a model.

**Parameters**:	
**keep_doctags_vectors** (bool) – Set keep_doctags_vectors to False if you don’t want to save doctags vectors, in this case you can’t to use docvecs’s most_similar, similarity etc. methods.
**keep_inference** (bool) – Set keep_inference to False if you don’t want to store parameters that is used for infer_vector method

You can find this piece of information in the documentation: https://radimrehurek.com/gensim/models/doc2vec.html

Briefly, it's used to trim unneeded model memory = use (much) less RAM. 

In [32]:
model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

## Saving the model (after deleting extra training data)

In [33]:
model.save("F:\\resulted_model")

# Done!